In [75]:
import os

import cv2
import keras
import keras.backend as K
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from keras.applications.resnet import (ResNet50, preprocess_input, decode_predictions)
from keras.applications.vgg16 import (VGG16, preprocess_input)
from tensorflow.python.framework import ops

In [76]:
import pandas as pd
import json

In [77]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


def load_image(path):
    img_path = path
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * tf.cast(op.inputs[0] > 0., dtype)


#def compile_saliency_function(model, activation_layer='bn5c_branch2b'):
def compile_saliency_function(model, activation_layer):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])


def deprocess_image(x):
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def modify_backprop_ResNet50(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = ResNet50(weights='imagenet')
    return new_model


def modify_backprop_VGG16(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model

In [78]:
model_res = ResNet50(weights='imagenet')
model_res.summary()
model_vgg = VGG16(weights='imagenet')
model_vgg.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [79]:
pic_folder = "./new_umap_image/"
pic_cam_folder_v = "./img_grad_cam/"
#pic_cam_folder_r = "./img_grad_cam_v/"
list_name = os.listdir(pic_folder)
print("number of image:", len(list_name))

# heat map HSV bounder setting
lower_blue = np.array([110, 100, 100])  #blue
upper_blue = np.array([130, 255, 255])

lower_green = np.array([40, 43, 46])  #green
upper_green = np.array([110, 255, 255])

lower_red = np.array([0, 43, 46])  #red
upper_red = np.array([34, 255, 255])

number of image: 300


In [80]:
def grad_cam(model, x, category_index, layer_name, model2, category_index2, layer_name2):
    """
    Args:
       model: model
       x: image input
       category_index: category index
       layer_name: last convolution layer name
    """
    # get category loss
    class_output = model.output[:, category_index]
    class_output2 = model2.output[:, category_index2]

    # layer output
    convolution_output = model.get_layer(layer_name).output
    convolution_output2 = model2.get_layer(layer_name2).output

    # get gradients
    grads = K.gradients(class_output, convolution_output)[0]
    grads2 = K.gradients(class_output2, convolution_output2)[0]

    # get convolution output and gradients for input
    gradient_function = K.function([model.input], [convolution_output, grads])
    output, grads_val = gradient_function([x])
    output, grads_val = output[0], grads_val[0]

    gradient_function2 = K.function([model2.input], [convolution_output2, grads2])
    output2, grads_val2 = gradient_function2([x])
    output2, grads_val2 = output2[0], grads_val2[0]

    # avg
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    weights2 = np.mean(grads_val2, axis=(0, 1))
    cam2 = np.dot(output2, weights2)

    # create heat map
    # model 1 Vgg16
    cam = cv2.resize(cam, (x.shape[1], x.shape[2]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)
    heatmap = np.where(heatmap > 0.35, heatmap, 0)
    cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    hsv = cv2.cvtColor(cam, cv2.COLOR_BGR2HSV)
    red_mask = cv2.inRange(hsv, lower_red, upper_red)
    red = cv2.bitwise_and(cam, cam, mask=red_mask)
    # model 2 ResNet
    cam2 = cv2.resize(cam2, (x.shape[1], x.shape[2]), cv2.INTER_LINEAR)
    cam2 = np.maximum(cam2, 0)
    heatmap2 = cam2 / np.max(cam2)
    heatmap2 = np.where(heatmap2 > 0.35, heatmap2, 0)
    cam2 = cv2.applyColorMap(np.uint8(255 * heatmap2), cv2.COLORMAP_WINTER)
    hsv2 = cv2.cvtColor(cam2, cv2.COLOR_BGR2HSV)
    green_mask2 = cv2.inRange(hsv2, lower_green, upper_green)
    green2 = cv2.bitwise_and(cam2, cam2, mask=green_mask2)

    # Return to BGR [0..255] from the preprocessed image
    image_rgb = x[0, :]
    image_rgb -= np.min(image_rgb)
    image_rgb = np.minimum(image_rgb, 255)

    # model_1
    cam_net1 = np.float32(red) + np.float32(image_rgb)
    # model_2
    cam_net2 = np.float32(green2) + np.float32(image_rgb)
    # combine model_1 and model_2
    cam_net3 = np.float32(red) + np.float32(green2) + np.float32(image_rgb)

    # color repair
    cam_net1 = 255 * cam_net1 / np.max(cam_net1)
    cam_net2 = 255 * cam_net2 / np.max(cam_net2)
    cam_net3 = 255 * cam_net3 / np.max(cam_net3)

    #print(cam)
    return np.uint8(cam_net1), np.uint8(cam_net2), np.uint8(cam_net3), heatmap, heatmap2


In [81]:
from keras.utils import img_to_array, load_img

##############################
# test function
##############################


file_name = list_name[1]
img = load_image(pic_folder + file_name)

# print function
# vgg 16 predictions
predictions_v = model_vgg.predict(img)
top_1_v = decode_predictions(predictions_v)[0][0]
# res 50 predictions
predictions_r = model_res.predict(img)
top_1_r = decode_predictions(predictions_r)[0][0]
print('Predicted class:')
print("pic_ID:", file_name)
print("vgg net :", '%s (%s) with probability %.2f' % (top_1_v[1], top_1_v[0], top_1_v[2]))
print("res net :", '%s (%s) with probability %.2f' % (top_1_r[1], top_1_r[0], top_1_r[2]))

predict_lable_model_1 = []
predict_lable_model_2 = []
prop_model_1 = []
prop_model_2 = []
predict_lable_model_1.append(top_1_v[1])
predict_lable_model_2.append(top_1_r[1])
prop_model_1.append(top_1_v[2])
prop_model_2.append(top_1_r[2])

predicted_class_v = np.argmax(predictions_v)
predicted_class_r = np.argmax(predictions_r)
cam_image1, cam_image2, cam_image3, heat_map_v, heat_map_r = grad_cam(model_vgg, img, predicted_class_v, "block1_conv1",
                                                                      model_res, predicted_class_r, "conv2_block1_1_relu")
img_file = load_img(pic_folder + file_name)
img_file = img_to_array(img_file)

# resize for cam_image
cam_image1 = cv2.resize(cam_image1, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
cam_image1 = cam_image1.astype('float32')
cam_image2 = cv2.resize(cam_image2, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
cam_image2 = cam_image2.astype('float32')
cam_image3 = cv2.resize(cam_image3, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
cam_image3 = cam_image3.astype('float32')

name_str = file_name.split('.')
name_heatmap_model_1 = name_str[0] + "_1" + "." + name_str[1]
name_heatmap_model_2 = name_str[0] + "_2" + "." + name_str[1]
name_heatmap_model_3 = name_str[0] + "_3" + "." + name_str[1]
cv2.imwrite(pic_cam_folder_v + name_heatmap_model_1, cam_image1)
cv2.imwrite(pic_cam_folder_v + name_heatmap_model_2, cam_image2)
cv2.imwrite(pic_cam_folder_v + name_heatmap_model_3, cam_image3)
print("save heatmap_model_1 => ", name_heatmap_model_1)
print("save heatmap_model_2 => ", name_heatmap_model_2)
print("save heatmap_combine => ", name_heatmap_model_3)
print("-------------------------------------------------------------")

Predicted class:
pic_ID: n01685808_123.JPEG
vgg net : whiptail (n01685808) with probability 0.79
res net : alligator_lizard (n01689811) with probability 0.85
save heatmap_model_1 =>  n01685808_123_1.JPEG
save heatmap_model_2 =>  n01685808_123_2.JPEG
save heatmap_combine =>  n01685808_123_3.JPEG
-------------------------------------------------------------


In [82]:
print(predicted_class_v)

41


In [ ]:
from keras.utils import image_utils

test_sort = []

predict_lable_model_1 = []
predict_lable_model_2 = []
prop_model_1 = []
prop_model_2 = []

model_1_heatmap_ID = []
model_2_heatmap_ID = []
combine_heatmap_ID = []

for i, file_name in enumerate(list_name):
    img = load_image(pic_folder + file_name)
    # print function
    # vgg 16 predictions
    predictions_v = model_vgg.predict(img)
    top_1_v = decode_predictions(predictions_v)[0][0]
    # res 50 predictions
    predictions_r = model_res.predict(img)
    top_1_r = decode_predictions(predictions_r)[0][0]
    print("Index:", i)
    print("File name:", file_name)
    print('Predicted class:')
    print("vgg net :", '%s (%s) with probability %.2f' % (top_1_v[1], top_1_v[0], top_1_v[2]))
    print("res net :", '%s (%s) with probability %.2f' % (top_1_r[1], top_1_r[0], top_1_r[2]))
    print("-------------------------------------------------------------")

    predict_lable_model_1.append(top_1_v[1])
    predict_lable_model_2.append(top_1_r[1])
    prop_model_1.append(str(top_1_v[2]))
    prop_model_2.append(str(top_1_r[2]))

    predicted_class_v = np.argmax(predictions_v)
    predicted_class_r = np.argmax(predictions_r)
    cam_image1, cam_image2, cam_image3, heat_map_v, heat_map_r = grad_cam(model_vgg, img, predicted_class_v,
                                                                          "block1_conv1",
                                                                          model_res, predicted_class_r, "conv2_block1_1_relu")
    img_file = image_utils.load_img(pic_folder + list_name[i])
    img_file = image_utils.img_to_array(img_file)

    # resize for cam_image
    cam_image1 = cv2.resize(cam_image1, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
    cam_image1 = cam_image1.astype('float32')
    cam_image2 = cv2.resize(cam_image2, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
    cam_image2 = cam_image2.astype('float32')
    cam_image3 = cv2.resize(cam_image3, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
    cam_image3 = cam_image3.astype('float32')

    name_str = file_name.split('.')
    name_heatmap_model_1 = name_str[0] + "_1" + "." + name_str[1]
    name_heatmap_model_2 = name_str[0] + "_2" + "." + name_str[1]
    name_heatmap_model_3 = name_str[0] + "_3" + "." + name_str[1]

    # create json data
    test_sort.append(name_str[0])
    model_1_heatmap_ID.append(name_str[0] + "_1")
    model_2_heatmap_ID.append(name_str[0] + "_2")
    combine_heatmap_ID.append(name_str[0] + "_3")

    cv2.imwrite(pic_cam_folder_v + name_heatmap_model_1, cam_image1)
    cv2.imwrite(pic_cam_folder_v + name_heatmap_model_2, cam_image2)
    cv2.imwrite(pic_cam_folder_v + name_heatmap_model_3, cam_image3)
    print("save heatmap_model_1 => ", name_heatmap_model_1)
    print("save heatmap_model_2 => ", name_heatmap_model_2)
    print("save heatmap_combine => ", name_heatmap_model_3)
    print("-------------------------------------------------------------")

Index: 0
File name: n01685808_102.JPEG
Predicted class:
vgg net : alligator_lizard (n01689811) with probability 0.62
res net : whiptail (n01685808) with probability 0.87
-------------------------------------------------------------
save heatmap_model_1 =>  n01685808_102_1.JPEG
save heatmap_model_2 =>  n01685808_102_2.JPEG
save heatmap_combine =>  n01685808_102_3.JPEG
-------------------------------------------------------------
Index: 1
File name: n01685808_123.JPEG
Predicted class:
vgg net : whiptail (n01685808) with probability 0.79
res net : alligator_lizard (n01689811) with probability 0.85
-------------------------------------------------------------
save heatmap_model_1 =>  n01685808_123_1.JPEG
save heatmap_model_2 =>  n01685808_123_2.JPEG
save heatmap_combine =>  n01685808_123_3.JPEG
-------------------------------------------------------------
Index: 2
File name: n01685808_147.JPEG
Predicted class:
vgg net : whiptail (n01685808) with probability 0.88
res net : whiptail (n016858

C:\Users\yuanh\AppData\Local\Temp\ipykernel_6800\1240100910.py:50: RuntimeWarning: invalid value encountered in divide
  heatmap2 = cam2 / np.max(cam2)


save heatmap_model_1 =>  n01685808_224_1.JPEG
save heatmap_model_2 =>  n01685808_224_2.JPEG
save heatmap_combine =>  n01685808_224_3.JPEG
-------------------------------------------------------------
Index: 8
File name: n01685808_58.JPEG
Predicted class:
vgg net : alligator_lizard (n01689811) with probability 0.66
res net : whiptail (n01685808) with probability 0.99
-------------------------------------------------------------
save heatmap_model_1 =>  n01685808_58_1.JPEG
save heatmap_model_2 =>  n01685808_58_2.JPEG
save heatmap_combine =>  n01685808_58_3.JPEG
-------------------------------------------------------------
Index: 9
File name: n01685808_81.JPEG
Predicted class:
vgg net : whiptail (n01685808) with probability 0.80
res net : banded_gecko (n01675722) with probability 0.38
-------------------------------------------------------------
save heatmap_model_1 =>  n01685808_81_1.JPEG
save heatmap_model_2 =>  n01685808_81_2.JPEG
save heatmap_combine =>  n01685808_81_3.JPEG
----------

In [ ]:
pic_name = os.listdir(pic_folder)
# store picture ID eg.n01685808_102 into pic_ID_list
pic_ID_list = []
# store picture label_ID eg.n01685808 into pic_label_ID_list
pic_label_ID_list = []
for i in range(len(pic_name)):
    temp = str.split(pic_name[i], '.')
    temp_1 = str.split(temp[0], '_')
    pic_ID_list.append(temp[0])
    pic_label_ID_list.append(temp_1[0])
#print(pic_ID_list)

In [ ]:
label_list = pd.read_csv("./list.txt", sep=' ', header=None)
label_value = label_list.values
ID = []
label_name = []
for i in range(30):
    ID.append(label_value[i][0])
    label_name.append(label_value[i][1])
label_dic = dict(zip(ID, label_name))
print("imageNet label dic shape:", label_value.shape)

pic_label_name_list = []
for i in range(len(pic_label_ID_list)):
    pic_label_name_list.append(label_dic[pic_label_ID_list[i]])

In [ ]:
# number of keys = 12
# picture_imageNet_ID = n01685808_165
# lable_ID = n01685808
# true_label = whiptail
# predict_lable_model_1 = default => 0
# predict_lable_model_2 = default => 0
# prop_model_1 = default => 0
# prop_model_2 = default => 0
# view_X = scatter x position(float)  default => 0
# view_Y = scatter y position(float)  default => 0
# model_1 heatmap ID
# model_2 heatmap ID
# combine heatmap ID


'''
predict_lable_model_1 = []
predict_lable_model_2 = []
prop_model_1 = []
prop_model_2 = []
test_sort = []
'''

In [ ]:
keys = ['picture_imageNet_ID', 'lable_ID', 'true_label',
        'predict_lable_model_1', 'prop_model_1',
        'predict_lable_model_2', 'prop_model_2',
        'view_X', 'view_Y',
        'model_1_heatmap_ID', 'model_2_heatmap_ID', 'combine_heatmap_ID']
# initial list setting, default value = 0

# T-SNE or UMap
view_X = np.zeros(300)
view_Y = np.zeros(300)

# keras data
#predict_lable_model_1 = np.zeros(300)
#prop_model_1 = np.zeros(300)
#predict_lable_model_2 = np.zeros(300)
#prop_model_2 = np.zeros(300)
#model_1_heatmap_ID = np.zeros(300)
#model_2_heatmap_ID = np.zeros(300)
#combine_heatmap_ID = np.zeros(300)


temp = []
temp_key = []
for i in range(300):
    temp_key.append(pic_ID_list[i])  # picture_imageNet_ID = n01685808_165
    temp_key.append(pic_label_ID_list[i])  # lable_ID = n01685808
    temp_key.append(pic_label_name_list[i])  # true_label = whiptail

    temp_key.append(predict_lable_model_1[i])  # predict_lable_model_1 = default => 0
    temp_key.append(prop_model_1[i])  # prop_model_1 = default => 0
    temp_key.append(predict_lable_model_2[i])  # predict_lable_model_2 = default => 0
    temp_key.append(prop_model_2[i])  # prop_model_2 = default => 0

    temp_key.append(view_X[i])  # view_X = scatter x position(float)  default => 0
    temp_key.append(view_Y[i])  # view_Y = scatter y position(float)  default => 0

    temp_key.append(model_1_heatmap_ID[i])  # model_1 heatmap ID
    temp_key.append(model_2_heatmap_ID[i])  # model_2 heatmap ID
    temp_key.append(combine_heatmap_ID[i])  # combine heatmap ID

    temp.append(temp_key)
    temp_key = []

list_json = [dict(zip(keys, item)) for item in temp]
str_json = json.dumps(list_json, indent=2, ensure_ascii=False)
filename = 'result_' + 'version_1' + '.json'
fileObject = open(filename, 'w')
fileObject.write(str_json)
fileObject.close()

In [ ]:
#print(temp)

In [ ]:
pic_label_name_list
predict_lable_model_1
predict_lable_model_2
error_count = 0
for i in range(300):

    if pic_label_name_list[i] != predict_lable_model_1[i] or pic_label_name_list[i] != predict_lable_model_2[i]:
        error_count += 1
        print("true:", pic_label_name_list[i])
        print("P1:", predict_lable_model_1[i])
        print("P2:", predict_lable_model_2[i])
print(error_count)

In [ ]:
with open(filename, 'r', encoding='utf8') as fp:
    json_data = json.load(fp)
print(json_data[1])
print(json_data[0]['picture_imageNet_ID'])

In [ ]:
'''
for i in range(300):
    if test_sort[i] == pic_ID_list[i]:
        print("success")
    else:
        print("failed")
'''